In [1]:
import pandas as pd
import numpy as np
import sys
import shutil
import glob
import os
import nibabel as nib
from pathlib import PosixPath
from scipy import ndimage
import matplotlib.pyplot as plt
from pathlib import Path
import random
import json
import itertools
import scipy
from scipy import stats
from scipy.io import loadmat
from scipy import ndimage
import sys
import matplotlib.image as mpimg
import skimage
import nibabel as nib
#import tensorflow as tf
#from tensorflow import keras
from nilearn.image import new_img_like
import cv2

In [2]:
def nifti_loader(filename):
    x = nib.load(filename)
    img = np.asanyarray(x.dataobj)
    #img = (img>0).astype(np.float32())
    
    return img

In [3]:
def smoothedDice(gt, pred):
    top_smooth = 1e-3
    smooth = 1e-3
    intersection = np.sum(gt * pred)
    denominator = np.sum((gt + pred))
    f = (2.0 * intersection + top_smooth) / (denominator + smooth)
    return f

In [182]:
def synthetic_cavity_annotator(t2_path,im_path):
    #load t2
    t2 = nifti_loader(t2_path)
    t2 = t2>0
    t2orig = nifti_loader(t2_path)
    t2orig = t2orig>0


    #dilate t2
    from skimage import morphology
    kern2 = np.ones((50,50,50))
    t2  = morphology.dilation(t2,selem=kern2)

    #close t2 
    kern1 = np.ones((12,12,12))
    t2  = morphology.closing(t2,selem=kern1)

    # read image
    image = nifti_loader(im_path)

    # norm image
    image = image + np.min(image)
    image = image / np.max(image)

    #calc threshold
    from skimage.filters import threshold_multiotsu
    thresh = threshold_multiotsu(image)
    print(thresh)

    #theshold and invert
    slic = image>thresh[0]
    slic = abs(1-slic)
    slic = np.asarray(slic,np.float32)

    #extract brain and close 
    from skimage import morphology
    kern3 = np.ones((30,30,30))
    closed  = morphology.opening(slic,selem=kern3)
    closed=abs(1-closed)

    kern4 = np.ones((18,18,14))
    closed_p  = morphology.erosion(closed,selem=kern4)
    
    #ensure edges in z are set to 0
    closed_p[:,:,150:]=0 
    closed_p[:,:,:40]=0 
    

    #----brain----
    # norm image
    image = image + np.min(image)
    image = image / np.max(image)

    #remove skull
    image1=image*closed_p

    #calc threshold above 
    from skimage.filters import threshold_multiotsu
    thresh = threshold_multiotsu(image1)

    #theshold
    slic = image1>(thresh[0])

    #ensure at least float 32 precision - convert float 32
    slic = np.asarray(slic,np.float32)

    #open to remove connections to skull
    kern4 = np.ones((2,2,2))
    closed  = morphology.erosion(slic,selem=kern4)
    
    #get just brain
    kern5 = np.ones((80,80,80))
    closed1  = morphology.closing(closed,selem=kern5)

    kern8=np.ones((5,5,5))
    yay=morphology.erosion(closed1,selem=kern8)
    
    #find top and bottom of brain
#     min_idx=0
#     sli=0
#     max_idx=np.shape(yay)[2]-1
#     while min_idx==0:
#         if np.sum(yay[:,:,sli])>2:
#             min_idx=sli
#         else:
#             sli+=1
#     sli=np.shape(yay)[2] -1
#     while max_idx==(np.shape(yay)[2]-1):
#         if np.sum(yay[:,:,sli])>2:
#             max_idx=sli
#         else:
#             sli-=1
    min_idx=0
    sli=0
    max_idx=np.shape(yay)[2]-1
    while min_idx==sli and np.sum(yay[:,:,sli])<2:
        if np.sum(yay[:,:,sli])>2:
            min_idx=sli
        else:
            sli+=1
            min_idx=sli
    sli=np.shape(yay)[2] -1
    while max_idx==sli and np.sum(yay[:,:,sli])<2:
        if np.sum(yay[:,:,sli])>2:
            max_idx=sli
        else:
            sli-=1
            max_idx=sli
    print(min_idx,max_idx)
    #apply brain
    closed = closed_p*yay
    #----------------------------------
    
    #create rough ventricle mask
    vent=np.zeros(np.shape(t2))

    #find center of the brain in middle slice
    z=int((min_idx+max_idx)/2)-21
    x1,y1=scipy.ndimage.measurements.center_of_mass(closed[:,:,z+5])
    x=int(x1)
    y=int(y1)

    vent[x-25:x+25,y-35:y+20,z+25:z+37]=1 #top top top
    vent[x-48:x+48,y-35:y+20,z+25:z+30]=1 #top top
    vent[x-32:x+32,y-60:y-20,z+10:z+25]=1 #top front
    vent[x-20:x+20,y-20:y+10,z+10:z+25]=1 #top middle
    vent[x-40:x+40,y+10:y+40,z+10:z+25]=1 #top back
    vent[x-34:x+34,y-70:y-30,z:z+10]=1 #middle front front
    vent[x-40:x+40,y-25:y+10,z-5:z+5]=1 #middle front
    vent[x-15:x+15,y-15:y+25,z:z+10]=1 #middle middle
    vent[x-35:x+35,y+20:y+55,z:z+10]=1 #middle back
    vent[x-20:x+20,y-30:y+40,z-15:z]=1 #bottom
    vent[x-42:x+42,y+20:y+68,z-15:z]=1 #bottom back
    vent[x-15:x+15,y-20:y+45,z-40:z-15]=1 #bottom bottom
    vent[x-25:x+25,y+20:y+45,z-40:z-30]=1 #bottom bottom
    vent[x-15:x+15,y-40:y,z-50:z-30]=1 #bottom curve front
    vent[x-15:x+15,y:y+30,:z-40]=1 #stem


    kern5 = np.ones((10,10,10))
    vent_s  = skimage.filters.gaussian(vent,8.5)
    vent_s=vent_s>.5

    #remove skull overlap
    selection=closed*t2
    selection=selection*abs(1-vent_s)
    sub_img=(selection*image) + abs(1-selection)


    #select all slices where there is t2 lesion
    min_idx=0
    sli=0
    max_idx=np.shape(t2orig)[2]-1
    while min_idx==sli and np.sum(t2orig[:,:,sli])<2:
        if np.sum(t2orig[:,:,sli])>2:
            min_idx=sli
        else:
            sli=sli+1
            min_idx=sli
    sli=np.shape(t2orig)[2] -1
    while max_idx==sli and np.sum(t2orig[:,:,sli])<2 :
        if np.sum(t2orig[:,:,sli])>2:
            max_idx=sli
        else:
            sli-=1
            max_idx=sli
    print(min_idx,max_idx)

    # use selection to threshold 
    sub_img[:,:,:min_idx]=0
    sub_img[:,:,max_idx:]=0
    b = np.where((sub_img > -1) & (sub_img < .4), True, False)
#     thresh2 = skimage.filters.threshold_multiotsu(sub_img[b],3)
    thresh2 = skimage.filters.threshold_otsu(sub_img[b])

    comp = sub_img<(thresh2)

    # open cavity mask
    kern = np.ones((3,3,3))
    closed2  = morphology.closing(comp,selem=kern)


    #define target overlap t2
    kern = np.ones((14,14,14))
    t2dil  = morphology.closing(t2orig,selem=kern)

    kern = np.ones((12,12,12))
    t2dil  = morphology.dilation(t2dil,selem=kern)

    #solve for most overlapping CC
    center_cav=np.zeros(np.shape(t2))
    for j in range(min_idx,max_idx):
        #Conencted component analysis
        labeled_image = skimage.measure.label(closed2[:,:,j], connectivity=1, return_num=True)
        dists=[]
        for k in range((np.max(labeled_image[0]))+1):
            dists.append(smoothedDice(labeled_image[0]==k,t2dil[:,:,j]))
        if len(dists)>1:
            sl_idx=dists.index(np.max(dists[1:]))
        else:
            sl_idx=0
        center_cav[:,:,j]=labeled_image[0]==sl_idx
    return center_cav.astype(np.float32),closed2.astype(np.float32)

In [83]:
t2path='/bogus/path/to/t2_lesion.nii.gz'
impath='/bogus/path/to/t2_image.nii.gz'
cavity,cc=synthetic_cavity_annotator(t2path,impath)

[0.12304688 0.32226562]
